In [ ]:
from numba import njit
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange # conda install -c conda-forge tqdm
%matplotlib notebook

rng = np.random.default_rng()

In [ ]:
N = 30
temp = 2.3
lattice = 2*rng.integers(2,size=(N,N)) - 1
pacc = np.array([np.exp(-dE/temp) for dE in range(-8,9,4)])

In [ ]:
def plotIsing(lattice):
    up   = np.where(lattice > 0)
    down = np.where(lattice < 0)
    plt.scatter(*up, c='red')
    plt.scatter(*down, c='blue')
    plt.gca().set_aspect(1)
    plt.xticks([]); plt.yticks([])
    plt.show()

In [ ]:
plotIsing(lattice)

In [ ]:
def energy(lattice):
    inds = np.arange(N**2)
    rows = inds//N
    cols = inds%N
    subLat1 = lattice[rows,cols]
    subLat2 = lattice[rows,(cols+1)%N]
    subLat3 = lattice[(rows+1)%N,cols]
    return -np.sum(subLat1*subLat2 + subLat1*subLat3)

In [ ]:
def magnetisation(lattice):
    return np.sum(lattice)

In [ ]:
jitted_magnetisation = njit()(magnetisation)

In [ ]:
jitted_magnetisation(lattice)

In [ ]:
%timeit magnetisation(lattice)
%timeit jitted_magnetisation(lattice)

In [ ]:
jitted_magnetisation(lattice)

In [ ]:
def mcstep(lattice, inds, rands):
    ii, jj = inds//N, inds%N
    for rand, i, j in zip(rands, ii, jj):
        site = lattice[i, j]
        deltaE = site*(lattice[i-1, j] + lattice[i, (j+1)%N] + lattice[(i+1)%N, j] + lattice[i, j-1])
        p = pacc[deltaE//2 + 2]
        if p >= 1.0:
            lattice[i, j] *= -1
        elif rand < p:
            lattice[i, j] *= -1
    return lattice

In [ ]:
def mcstep2(lattice, inds, rands):
    ii, jj = inds//N, inds%N
    for k in range(len(rands)):
        site = lattice[ii[k], jj[k]]
        deltaE = site*(lattice[ii[k]-1, jj[k]] + lattice[ii[k], (jj[k]+1)%N] + lattice[(ii[k]+1)%N, jj[k]] + lattice[ii[k], jj[k]-1])
        p = pacc[deltaE//2 + 2]
        if p >= 1.0:
            lattice[ii[k], jj[k]] *= -1
        elif rands[k] < p:
            lattice[ii[k], jj[k]] *= -1
    return lattice

In [ ]:
jitted_mcstep = njit()(mcstep)
jitted_mcstep2 = njit()(mcstep2)

In [ ]:
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
jitted_mcstep2(lattice, inds, rands)

In [ ]:
%%timeit
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
mcstep(lattice, inds, rands)

In [ ]:
%%timeit
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
jitted_mcstep(lattice, inds, rands)

In [ ]:
%%timeit
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
jitted_mcstep2(lattice, inds, rands)

In [ ]:
inds = rng.integers(N**2, size=(N**2))
rands= rng.random(N**2)
mcstep(lattice, inds, rands) == jitted_mcstep(lattice, inds, rands)